In [ ]:
import torch
import utils 
import math
import random
import helpers
import numpy as np 
import matplotlib as mlp
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder

# some useful functions

In [ ]:
def compute_cvPCA(resp0):
    ss0 = utils.shuff_cvPCA(resp0, nshuff=10)
    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0

## Alexnet Model 

* get model architecture:

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

# define hook for intermediate activations

In [ ]:
num_rois = 193600
num_stimulus = 2800
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def ghook(model, input, output):
    return intermediate_activations.append(output)

intermediate_activations = []

arr00 = np.empty((num_stimulus, num_rois), dtype=float)


## Using subset of ImageNet data, define transforms on data

In [ ]:
batch_size = 1

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
         
            ]
        )

imagenet_data = ImageFolder('/Users/duuta/ppp/notebooks/robustSigs/ImageNet_data/imagenet-mini/', transform=transform)
imagenet_data2800 = Subset(imagenet_data, np.arange(2800))
data_loader = torch.utils.data.DataLoader(imagenet_data2800, 
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=10) 

num_epochs = int(len(imagenet_data2800)/ batch_size)

In [ ]:
# features_list = [0, 3, 6, 8, 10]
# number of neurons 193600 
# set batch size and scale with 

In [ ]:
sample_target_layer = model.features[0]
for epoch  in range(1000):
    for j, (input, _) in enumerate(iter(data_loader)):
        hook_handle = sample_target_layer.register_forward_hook(get_activation('feats'))
        _ = model(input)
        arr00[j, :] = activation['feats'].reshape(1, num_rois)

hook_handle.remove()

* move imagenet data to nemo, computation cost is 1min/image 

In [ ]:
arr00_image2K = arr00[:2800, :]

In [ ]:
with mlp.rc_context({'figure.figsize': (20, 30)}):
    plt.matshow(arr00_image2K)

In [ ]:
fig0 = plt.figure(figsize=(20, 10))
fig0.suptitle('Showing response of AlexNet ROIs in Layer 1')
ax = fig0.add_subplot()
plt.imshow(arr00_image2K)
ax.set_xlabel('ROIs')
jax.set_ylabel('Stimulus Number')

In [ ]:
response = helpers.dupSignal(arr00_image2K, istm0)

In [ ]:
response.shape

In [ ]:
# create (2 * stim * neurons) 
# arr0  = helpers.ssplit(arr00_image2K)

In [ ]:
def ssplit0(x):
    """Returns a new array of split
    spontaneous activity.
    (2, n, p)
    """
    # get # of stim
    nr = x.shape[0]
    snr = math.ceil(nr / 2)

    # get # neurons
    nco = x.shape[1]

    # init empty for final results
    newarr = np.empty((2, snr, nco))

    p0 = x[:snr, :]
    p1 = x[snr:nr, :]

    # store data
    newarr[0, :, :] = p0
    newarr[1, :, :] = p1

    return newarr

In [ ]:
arr0 = ssplit0(arr00_image2K)

In [ ]:
ss0 =compute_cvPCA(arr0)

In [ ]:
ss0_ = compute_cvPCA(response)

In [ ]:
ss0_.shape

In [ ]:
alpha0, ypred0, _ = utils.get_powerlaw(ss0_/ss0.sum(), np.arange(11, 5e2).astype('int'))

In [ ]:
alpha0

# Using stimulus index (like in stringer)

In [ ]:
with mlp.rc_context({'figure.figsize':(8, 6)}):
    
    plt.loglog(np.arange(0, ss0_.size)+1, ss0_/ss0_.sum())
    plt.loglog(np.arange(0, ss0_.size)+1, ypred0)
    plt.title("Plot of Variance drop Off by dimensions")
    plt.xlabel('Dimensions')
    plt.ylabel('Variance')

# not using stringer computation 

In [ ]:
ss00 = np.array(sorted(ss0))

In [ ]:
alpha_value, ypred , _= utils.get_powerlaw(ss0/ss0.sum(), np.arange(11, 5e2).astype('int'))

In [ ]:
alpha_value


In [ ]:
with mlp.rc_context({'figure.figsize': (8, 6)}):
    plt.loglog(np.arange(0, ss0.size)+1, ss0/ss0.sum())
    plt.loglog(np.arange(0, ss0.size)+1, ypred)


In [ ]:
istm0 = np.concatenate((list(range(0, 1400)), list(range(0, 1400))))

* need to figure out dupSignal function, seems repeats responses are sought and averaged
* created stim index array [0, 1399]*2 in the other in which the stimulus was presented 
* results were not different from what I got splitting the data with ssplit. Also, alpha values were low 0.4777, 0.482,
* need to figure out what happens... NB: in stringer experiment repeat stimulus in alexnet, there are no repeats. 8
* these are important to create a 2* N * M stimulus and then compute cvPCA.
* I could pretend this is spontaneous activity and split the signal then procced
* extend the computation to next layers
* 